# Configuración para la arquitectura del proyecto

## Collection de Amazon Rekognition

Primero se crea una coleccion en amazon rekognition para almacenar la cara de las personas que serán identificadas posteriormente, amazon solo comparara las imagenes entrantes al sistema contra aquellas almacenadas en la collection 

In [10]:
import boto3

# La función create_collection, crea la collection 
# recibe el nombre de la collection que se desea crear 

def create_collection(collection_id):

    #Se hace el llamdo al cliente de amazon rekognition 
    client=boto3.client('rekognition')

    #Creación de la collection
    response=client.create_collection(CollectionId=collection_id)
    print('Collection ' + collection_id + ' creada')
    
def main():
    collection_id='collection-epp'
    create_collection(collection_id)

if __name__ == "__main__":
    main()  

Collection collection-epp creada


Se añaden las caras a la collection, se da el path del local o el bucket donde se encuentran las imagenes que se desean agregar a la collection, principalmente estas imagenes tendran un faceId, ImageID y externalImageID, en la collection

In [ ]:
import boto3
import os

#En este caso agregamos las imagenes desde el path del local 
path =r'C:\Users\user\Desktop\collection'
fpath = os.listdir(path)

diccionario = {}

#la funcion  add_faces_to_collection añade una cara a la collection
#recibe como parametro la imagen, el nombre de la imagen, la collection_id 
#donde se desea agregar la cara y la region de aws

def add_faces_to_collection(target_file, photo, collection_id, region):
    # Se llama al cliente de rekognition en la region indicada 
    client = boto3.client('rekognition', region_name=region)

    #Abrimos la imagen 
    imageTarget = open(target_file, 'rb')
    
    # con la funcion index_faces agregamos la imagen en bytes a la collection indicada
    response = client.index_faces(CollectionId=collection_id,
        Image={'Bytes': imageTarget.read()},
        ExternalImageId=photo,
        MaxFaces=1,
        QualityFilter="AUTO")
    

    #Creamos un diccionario con el FaceId de la persona indexada y valor de path de imagen
    diccionario[response['FaceRecords'][0]['Face']['FaceId']] = images
    
# se crea una dos ciclos for para recorrer el path del local y cada directorio en el, donde se encuentran las imagenes
for folder in fpath:
    path_folders =  path +'\\' + folder
    fpath_folders = os.listdir(path_folders)
    for images in fpath_folders:
        image = r''+path + '\\'+ folder + '\\'+ images
        collection_id = 'collection-epp'
        photo_name = folder
        region = "us-east-1"
                                        
        indexed_faces_count = add_faces_to_collection(image, photo_name, collection_id, region)

print("Se agregaron correctamente todas las imagenes")

Se realiza una descripcion de la collection y así verificar que fueron bien suubidas las imagenes 

In [2]:
import boto3
from botocore.exceptions import ClientError

# La función describe_collection permite tener la información de la collection
# recibe como parametro el nombre de la collection 
def describe_collection(collection_id):
    #Se hace el llamdo al cliente de rekognition
    client=boto3.client('rekognition')

    try:
        #La funcion describe_collection retorna un json con la información de la collection
        #imrpimimos aquella informacion que nos interese
        response=client.describe_collection(CollectionId=collection_id)
        print("Collection Arn: "  + response['CollectionARN'])
        print("Face Count: "  + str(response['FaceCount']))
        print("Face Model Version: "  + response['FaceModelVersion'])
        print("Timestamp: "  + str(response['CreationTimestamp']))

        # En dado caso que la collection no exista saldra un mensaje de error
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print ('The collection ' + collection_id + ' was not found ')
        else:
            print ('Error other than Not Found occurred: ' + e.response['Error']['Message'])
    print('Done...')


def main():
    collection_id='collection-rekognition'
    describe_collection(collection_id)

if __name__ == "__main__":
    main()

Collection Arn: arn:aws:rekognition:us-east-1:853702706419:collection/collection-rekognition
Face Count: 15
Face Model Version: 6.0
Timestamp: 2022-08-10 13:47:28.168000-05:00
Done...


Las caras en la collection se guardan en un formato json, puede ser de interes consultar la informacion de estas caras en la collection 

In [1]:
import boto3

lista = []

# la funcion list_faces_in_collection retorna una lista con la información de cada
# cara en la collection
#  
def list_faces_in_collection(collection_id):


    maxResults=1
    faces_count=0
    tokens=True

    # se invoca el cliente de amazon rekognition
    client=boto3.client('rekognition')
    
    # la funcion list_faces retorna una lista de todos los json de las caras en la collection
    response=client.list_faces(CollectionId=collection_id,
                               MaxResults=maxResults)

    # creamos un ciclo para recorrer la lista de los json 
    while tokens:

        list1 = []
        faces=response['Faces']

        # A cada face en la collection, extraeremos los datos de FaceId, ImageId
        # ExternalImageId y los agregamos a una lista 
        for face in faces:
            list1.append(face['FaceId'])
            list1.append(face['ImageId'])
            list1.append(face['ExternalImageId'])
            lista.append(list1)
            list1.clear

            faces_count+=1
        if 'NextToken' in response:
            nextToken=response['NextToken']

            response=client.list_faces(CollectionId=collection_id,
                                       NextToken=nextToken,MaxResults=maxResults)
        else:
            tokens=False
    return faces_count   
def main():

    collection_id='collection-rekognition'
    faces_count=list_faces_in_collection(collection_id)
    print("faces count: " + str(faces_count))
    
if __name__ == "__main__":
    main()

faces count: 15


## DynamoDB

Creamos una tabla en dynamoDB que nos permite almacenar la información de las personas agregadas en la collection, para ello se toma como primary_key el faceId de cada persona en la collection

In [ ]:
from pprint import pprint
import boto3

# la función create_dynamodb_table crea la base de datos en dynamoDb 
# la cual recibe como parametro el nombre de la tabla y la region de aws

# Create DynamoDB database with image URL and face data, face ID
def create_dynamodb_table(table_name, region):
    # se llama al cliente de dynamoDb
    dynamodb = boto3.client("dynamodb", region_name=region)

    # la funcion create_table, crea la tabla con primary_key igual al faceId 
    table = dynamodb.create_table(
        TableName=table_name,   
        KeySchema=[{
                'AttributeName': 'FaceID', 'KeyType': 'HASH'  # Partition key  
                },],        
            AttributeDefinitions=[
            {
                'AttributeName': 'FaceID', 'AttributeType': 'S'  }, ],        
                ProvisionedThroughput={
            'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10  }
    )
    pprint(table)
    return table

region = "us-east-1"
database_name = 'dataset-collection-images'
dynamodb_table = create_dynamodb_table(database_name, region)
print("Table status:", dynamodb_table)

Es una buena práctica las imagenes que agregamos a la collection tenerlas en un bucket de s3, por si en algun momento se requiere visualizar y tener el archivo .jpg de las personas en la collection

In [ ]:
import boto3
import logging
from botocore.exceptions import ClientError

# se recorre el path del local y los directorios donde estan las imagenes en dos ciclos 
# para subir imagen por imagen al bucket de s3
for folder in fpath:
    path_folders =  path +'\\' + folder
    fpath_folders = os.listdir(path_folders)
    for images in fpath_folders:
        file_name = r''+path + '\\'+ folder + '\\'+ images
        # store local file in S3 bucket
        bucket = "prueba-bucket-machine"
        key_name = folder+'/'+ images
        region = "us-east-1"

        # se llama al cliente de s3 
        s3 = boto3.client('s3', region_name=region)
        # Upload the file
        try:
            #La función upload_file actualiza y sube la imagen al bucket
            response = s3.upload_file(file_name, bucket, key_name)
        except ClientError as e:
            logging.error(e)
            
print("se logro subir correctamente las imagenes al bucket")

Se crea un código que permita obtener todos los objetos del bucket y ponerlos en una lista 

In [ ]:
# se invoca el servicio de amazon s3
client = boto3.client('s3')

# nombre del bucket
bucketName = "prueba-bucket-machine"

keys = []

# la funcion list_objects retorna un json con todos los objetos en el bucket
response = client.list_objects(
            Bucket=bucketName,
            MaxKeys=123,
        )
# recorremos cada uno de estos paths y los agregamos a una lista
for key in response['Contents']:
    keys.append(key['Key'])

print("Se agregaron los paths de los objetos a una lista")

Agregar los datos de las caras de la collection a dynamodb y agregar la url de las imagenes de s3 como una etiqueta en dynamodb 

In [ ]:
import boto3
from pprint import pprint
from decimal import Decimal
import json
#Nombre del bucket donde se encuentran las imagenes
bucket = "prueba-bucket-machine"

# la lista creada en la descripcion de las caras en la collection y los paths en la lista keys se recorren
# y se emparejan los faceId de cada cara con la imagen en el bucket
for item in lista:
    for key in keys:
        #sabemos que en ambas lista coincide el nombre de la carpeta en el bucket con el ExternalImageId de lista
        list = key.split('/')
        if list[0]==item[2]:
            # The local file that was stored in S3 bucket
            file_name = '/'+ diccionario[item[0]]
            key_name = key
            region = "us-east-1"
            # Get URL of file
            file_url = "https://s3.amazonaws.com/{}/{}".format(bucket, key_name)

            # upload face-id, face info, and image url
            def AddDBEntry(name, file_url, face_id, image_id,status):
                #se invoca el servicio de dynamoDb
                dynamodb = boto3.resource('dynamodb', region_name=region)
                #se da el nombre de la tabla creada
                table = dynamodb.Table('dataset-collection-images')
                # Se lanza el item o el dato de cada persona en la collection 
                response = table.put_item(
                Item={
                        'ExternalImageID': name,
                        'ImageURL': file_url,
                        'FaceID': face_id,
                        'ImageID': image_id, 
                        'status': status
                }
                )
                return response

            # Mock values for face ID, image ID, and confidence - replace them with actual values from your collection results
            try:
                dynamodb_resp = AddDBEntry(item[2], file_url, item[0],  
                item[1],False)
            except:
                print("El usuario ya se encuentra en la tabla")
            
print("Se agrego exitosamente los datos de la collection a dynamodb")

Creamos una segunda tabla en dynamo db la cual tendra la asistencia y datos personales de cada una de las personas en la collection 

In [1]:
from pprint import pprint
import boto3

# la función create_dynamodb_table crea la base de datos en dynamoDb 
# la cual recibe como parametro el nombre de la tabla y la region de aws

def create_dynamodb_table(table_name, region):
    # se llama al cliente de dynamoDb
    dynamodb = boto3.client("dynamodb", region_name=region)

    # la funcion create_table, crea la tabla con primary_key igual al Nombre
    table = dynamodb.create_table(
        TableName=table_name,   
        KeySchema=[{
                'AttributeName': 'Nombre', 'KeyType': 'HASH'  # Partition key  
                },],        
            AttributeDefinitions=[
            {
                'AttributeName': 'Nombre', 'AttributeType': 'S'  }, ],        
                ProvisionedThroughput={
            'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10  }
    )
    pprint(table)
    return table

region = "us-east-1"
database_name = 'dataset-collection-personal'
dynamodb_table = create_dynamodb_table(database_name, region)
print("Table status:", dynamodb_table)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '524',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Wed, 17 Aug 2022 15:32:11 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '4126457422',
                                      'x-amzn-requestid': 'U3QH85CAQQFJQ00VRABQ1FFLTBVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'U3QH85CAQQFJQ00VRABQ1FFLTBVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0},
 'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'Nombre',
                                                'AttributeType': 'S'}],
                      'CreationDateTime': datetime.datetime(2022, 8, 17, 10, 32, 11, 403000, tzinfo=tzlocal()),
                      'ItemCou

Agregamos el personal a la base de datos del personal

In [9]:
import boto3
from pprint import pprint
from decimal import Decimal
import json

# upload face-id, face info, and image url
def AddDBEntry(name, fecha,hora,status):
    #se invoca el servicio de dynamoDb
    dynamodb = boto3.resource('dynamodb', region_name=region)
    #se da el nombre de la tabla creada
    table = dynamodb.Table('dataset-collection-personal')
    # Se lanza el item o el dato de cada persona en la collection 
    response = table.put_item(
    Item={
        'Nombre': name,
        'Fecha': fecha,
        'Hora' : hora,
        'status': status
        }
    )
    return response
# la lista creada en la descripcion de las caras en la collection y los paths en la lista keys se recorren
# y se emparejan los faceId de cada cara con la imagen en el bucket
personas = []
for item in lista:
    # Mock values for face ID, image ID, and confidence - replace them with actual values from your collection results
    if item[2] not in personas: 
        personas.append(item[2])
        try:
            dynamodb_resp = AddDBEntry(item[2], '17-08-2022', '18:25 ',False)
        except:
            print("El usuario ya se encuentra en la tabla")
            
print("Se agrego exitosamente los datos de la collection a dynamodb")

Se agrego exitosamente los datos de la collection a dynamodb


## EPP

creamos la collection para las Epp 

In [ ]:
import boto3

# La función create_collection, crea la collection 
# recibe el nombre de la collection que se desea crear 

def create_collection(collection_id):

    #Se hace el llamdo al cliente de amazon rekognition 
    client=boto3.client('rekognition')

    #Creación de la collection
    response=client.create_collection(CollectionId=collection_id)
    print('Collection ' + collection_id + ' creada')
    
def main():
    collection_id='collection-epp'
    create_collection(collection_id)

if __name__ == "__main__":
    main()  

Agregamos las personas a la collection, en este caso lo hacemos manual y una imagen por persona en la collecion 

In [ ]:
import boto3

def add_faces_to_collection(target_file, nombre, collection_id, region):
    client = boto3.client('rekognition', region_name=region)

    imageTarget = open(target_file, 'rb')

    response = client.index_faces(CollectionId=collection_id,
                                  Image={'Bytes': imageTarget.read()},
                                  ExternalImageId=nombre,
                                  MaxFaces=1,
                                  QualityFilter="AUTO",
                                  DetectionAttributes=['ALL'])

    print('Results for ' + nombre)
    print('Faces indexed:')
    for faceRecord in response['FaceRecords']:
        print('  Face ID: ' + faceRecord['Face']['FaceId'])
        print('  Location: {}'.format(faceRecord['Face']['BoundingBox']))
        print('  Image ID: {}'.format(faceRecord['Face']['ImageId']))
        print('  External Image ID: {}'.format(faceRecord['Face']['ExternalImageId']))
        print('  Confidence: {}'.format(faceRecord['Face']['Confidence']))

    print('Faces not indexed:')
    for unindexedFace in response['UnindexedFaces']:
        print(' Location: {}'.format(unindexedFace['FaceDetail']['BoundingBox']))
        print(' Reasons:')
        for reason in unindexedFace['Reasons']:
            print('   ' + reason)
    return len(response['FaceRecords'])

image = r'C:\Users\user\Desktop\Anibal.jpg'
collection_id = 'collection-epp'
photo_name = 'Anibal_Gaviria'
region = "us-east-1"

indexed_faces_count = add_faces_to_collection(image, photo_name, collection_id, region)
print("Faces indexed count: " + str(indexed_faces_count))

Creamos la tabla de dynamodB con la collection de las personas y el faceId de cada persona como identificador en la tabla

In [ ]:
from pprint import pprint
import boto3

# la función create_dynamodb_table crea la base de datos en dynamoDb 
# la cual recibe como parametro el nombre de la tabla y la region de aws

def create_dynamodb_table(table_name, region):
    # se llama al cliente de dynamoDb
    dynamodb = boto3.client("dynamodb", region_name=region)

    # la funcion create_table, crea la tabla con primary_key igual al Nombre
    table = dynamodb.create_table(
        TableName=table_name,   
        KeySchema=[{
                'AttributeName': 'faceId', 'KeyType': 'HASH'  # Partition key  
                },],        
            AttributeDefinitions=[
            {
                'AttributeName': 'faceId', 'AttributeType': 'S'  }, ],        
                ProvisionedThroughput={
            'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10  }
    )
    pprint(table)
    return table

region = "us-east-1"
database_name = 'dataset-collection-epp'
dynamodb_table = create_dynamodb_table(database_name, region)
print("Table status:", dynamodb_table)

In [4]:
import boto3

lista = []

# la funcion list_faces_in_collection retorna una lista con la información de cada
# cara en la collection
#  
def list_faces_in_collection(collection_id):


    maxResults=1
    faces_count=0
    tokens=True

    # se invoca el cliente de amazon rekognition
    client=boto3.client('rekognition')
    
    # la funcion list_faces retorna una lista de todos los json de las caras en la collection
    response=client.list_faces(CollectionId=collection_id,
                               MaxResults=maxResults)

    # creamos un ciclo para recorrer la lista de los json 
    while tokens:

        list1 = []
        faces=response['Faces']

        # A cada face en la collection, extraeremos los datos de FaceId, ImageId
        # ExternalImageId y los agregamos a una lista 
        for face in faces:
            list1.append(face['FaceId'])
            list1.append(face['ImageId'])
            list1.append(face['ExternalImageId'])
            lista.append(list1)
            list1.clear

            faces_count+=1
        if 'NextToken' in response:
            nextToken=response['NextToken']

            response=client.list_faces(CollectionId=collection_id,
                                       NextToken=nextToken,MaxResults=maxResults)
        else:
            tokens=False
    return faces_count   
def main():

    collection_id='collection-epp'
    faces_count=list_faces_in_collection(collection_id)
    print("faces count: " + str(faces_count))
    
if __name__ == "__main__":
    main()
print(lista)

faces count: 2
[['cbdc9c93-94b6-4b14-90e6-7ecce1d656f0', '02641a6d-3f87-3657-9b73-daf314f316fa', 'Anibal_Gaviria'], ['fc9c0d99-a3d8-4c69-86b1-a4c0a290a7a2', '97ca740d-d263-3f5d-bab3-7550eac1e4dc', 'Daniel_Quintero_Calle']]


Agregamos las personas de la collection a la base de datos de dynamodB

In [7]:
import boto3
from pprint import pprint
from decimal import Decimal
import json

# upload face-id, face info, and image url
def AddDBEntry(faceId,nombre,casco,guantes,tapabocas,fecha,hora):
    #se invoca el servicio de dynamoDb
    dynamodb = boto3.resource('dynamodb', region_name=region)
    #se da el nombre de la tabla creada
    table = dynamodb.Table('dataset-collection-epp')
    # Se lanza el item o el dato de cada persona en la collection 
    response = table.put_item(
    Item={
        'faceId': faceId,
        'Nombre': nombre,
        'Casco' : casco,
        'Guantes': guantes,
        'TapaBocas': tapabocas,
        'Fecha': fecha,
        'Hora' : hora,
        }
    )
    return response
# la lista creada en la descripcion de las caras en la collection y los paths en la lista keys se recorren
# y se emparejan los faceId de cada cara con la imagen en el bucket
for item in lista:
    # Mock values for face ID, image ID, and confidence - replace them with actual values from your collection results
    
    try:
        dynamodb_resp = AddDBEntry(item[0], item[2].replace('_',' '), False,False,False, '2022-08-11' ,'18:25 ')
    except:
        print("El usuario ya se encuentra en la tabla")
            
print("Se agrego exitosamente los datos de la collection a dynamodb")

Se agrego exitosamente los datos de la collection a dynamodb
